## Level 12 Data Processing and ETL

---

### data cleaning --->

In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame({
    "Name": ["Mariz", "Luka", "Chris", np.nan],
    "Age": [25, 27, 44, 35],
    "occupation": ["Engineer", "Competitive Eater", "Artist", "Scientist"]
})

In [7]:
df.isna()

,Name,Age,occupation
0,False,False,False
1,False,False,False
2,False,False,False
3,True,False,False


In [10]:
df.dropna(inplace=True)
df

,Name,Age,occupation
0,Mariz,25,Engineer
1,Luka,27,Competitive Eater
2,Chris,44,Artist


In [11]:
df = pd.DataFrame({
    "Transaction_Amount": [100, 150, 200, 25, 300, 9999]
})

In [12]:
df

,Transaction_Amount
0,100
1,150
2,200
3,25
4,300
5,9999


In [14]:
from scipy import stats

In [16]:
df['z_score'] = stats.zscore(df['Transaction_Amount'])
df

,Transaction_Amount,z_score
0,100,-0.462083
1,150,-0.448457
2,200,-0.434832
3,25,-0.482521
4,300,-0.407581
5,9999,2.235473


In [18]:
df['Winsorized_Amount'] = stats.mstats.winsorize(df['Transaction_Amount'], limits=[0.05, 0.05])
df

,Transaction_Amount,z_score,Winsorized_Amount
0,100,-0.462083,100
1,150,-0.448457,150
2,200,-0.434832,200
3,25,-0.482521,25
4,300,-0.407581,300
5,9999,2.235473,9999


In [20]:
threshold = 500

df_truncate = df[df['Transaction_Amount'] <= threshold]
df_truncate

,Transaction_Amount,z_score,Winsorized_Amount
0,100,-0.462083,100
1,150,-0.448457,150
2,200,-0.434832,200
3,25,-0.482521,25
4,300,-0.407581,300


---
### label encoder & data transformation --->

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
df = pd.DataFrame({"Rating": ["Low", "Medium", "High", "Medium", "Low"]})
df

,Rating
0,Low
1,Medium
2,High
3,Medium
4,Low


In [25]:
le = LabelEncoder()

In [26]:
df["Encoded_Ratings"] = le.fit_transform(df['Rating'])
df

,Rating,Encoded_Ratings
0,Low,1
1,Medium,2
2,High,0
3,Medium,2
4,Low,1


In [28]:
pd.get_dummies(df['Rating'], drop_first=False)

,High,Low,Medium
0,False,True,False
1,False,False,True
2,True,False,False
3,False,False,True
4,False,True,False


In [31]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
df = pd.DataFrame({"Value": [10, 20, 30, 40, 50]})

In [32]:
scaler = MinMaxScaler()

In [34]:
df['Scaled_Values'] = scaler.fit_transform(df['Value'].values.reshape(-1, 1))
df

,Value,Scaled_Values
0,10,0.00
1,20,0.25
2,30,0.50
3,40,0.75
4,50,1.00


In [36]:
df = pd.DataFrame({"age":[22, 35, 55, 42, 68]})
df

,age
0,22
1,35
2,55
3,42
4,68


In [37]:
bins = [0, 30, 50, 100]
labels = ['Young', 'Middle-aged', 'Senior']

In [39]:
df['bin_age'] = pd.cut(df['age'], bins=bins, labels=labels)
df

,age,bin_age
0,22,Young
1,35,Middle-aged
2,55,Senior
3,42,Middle-aged
4,68,Senior


In [40]:
from sklearn.preprocessing import PolynomialFeatures

In [41]:
df = pd.DataFrame({"size": [550, 700, 900], "rooms":[2, 3, 4]})

In [43]:
poly_f = PolynomialFeatures(degree=2, include_bias=False)

In [46]:
df_poly = pd.DataFrame(poly_f.fit_transform(df), columns=poly_f.get_feature_names_out(df.columns))
df_poly

,size,rooms,size^2,size rooms,rooms^2
0,550.0,2.0,302500.0,1100.0,4.0
1,700.0,3.0,490000.0,2100.0,9.0
2,900.0,4.0,810000.0,3600.0,16.0


---
### handling imbalanced data --->

In [47]:
from sklearn.utils import resample

In [48]:
df = pd.DataFrame({"class":[0, 0, 1, 0, 0, 1, 0], "features":[1, 2, 3, 4, 5, 6, 7]})

In [49]:
df_majority = df[df["class"] == 0]

In [50]:
df_majority

,class,features
0,0,1
1,0,2
3,0,4
4,0,5
6,0,7


In [51]:
df_minority = df[df["class"] == 1]
df_minority

,class,features
2,1,3
5,1,6


In [54]:
df_undersampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_undersampled

,class,features
1,0,2
6,0,7


In [56]:
balanced_data = pd.concat([df_minority, df_undersampled])
balanced_data

,class,features
2,1,3
5,1,6
1,0,2
6,0,7


In [61]:
df_oversampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=515)
df_oversampled

,class,features
5,1,6
2,1,3
5,1,6
2,1,3
2,1,3
